In [ ]:
!pip install ipywidgets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.4/139.4 kB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 61.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 214.9/214.9 kB 30.0 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.0.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip


In [ ]:
import pandas as pd
import ipywidgets

# Load the data
file_path = 'cab_rides.csv'
cab_rides_df = pd.read_csv(file_path)

# Display the first few rows of the dataframe
cab_rides_df.head()


,distance,cab_type,time_stamp,destination,source,price,surge_multiplier,id,product_id,name
0,0.44,Lyft,1544952607890,North Station,Haymarket Square,5.0,1.0,424553bb-7174-41ea-aeb4-fe06d4f4b9d7,lyft_line,Shared
1,0.44,Lyft,1543284023677,North Station,Haymarket Square,11.0,1.0,4bd23055-6827-41c6-b23b-3c491f24e74d,lyft_premier,Lux
2,0.44,Lyft,1543366822198,North Station,Haymarket Square,7.0,1.0,981a3613-77af-4620-a42a-0c0866077d1e,lyft,Lyft
3,0.44,Lyft,1543553582749,North Station,Haymarket Square,26.0,1.0,c2d88af2-d278-4bfd-a8d0-29ca77cc5512,lyft_luxsuv,Lux Black XL
4,0.44,Lyft,1543463360223,North Station,Haymarket Square,9.0,1.0,e0126e1f-8ca9-4f2e-82b3-50505a09db9a,lyft_plus,Lyft XL


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from ipywidgets import interact, IntSlider

class DynamicPricing:
    def __init__(self, initial_eta_mean, initial_eta_std, a):
        self.eta_mean = initial_eta_mean
        self.eta_std = initial_eta_std
        self.a = a
        self.price_points = []
        self.eta_points = []
        self.sampled_etas = []
        self.actual_prices = []
        self.revenue_history = []
        self.cumulative_revenue = 0  # Initialize cumulative revenue

    def update_posterior(self, price, actual_price, demand):
        # Ensure standard deviation is never zero
        if self.eta_std < 1e-6:
            self.eta_std = 1e-6

        # Calculate the sample eta based on actual price and demand
        eta_sample = -np.log(demand / self.a) / np.log(price)
        
        # Use a fixed value for likelihood_std
        likelihood_std = 0.1  # This can be adjusted as needed for the model

        # Bayesian update
        new_mean_numerator = self.eta_mean / self.eta_std**2 + eta_sample / likelihood_std**2
        new_mean_denominator = 1 / self.eta_std**2 + 1 / likelihood_std**2
        new_mean = new_mean_numerator / new_mean_denominator if new_mean_denominator != 0 else self.eta_mean
        
        new_std = (1 / self.eta_std**2 + 1 / likelihood_std**2)**(-0.5) if new_mean_denominator != 0 else self.eta_std

        # Logging detailed updates
        old_eta_mean, old_eta_std = self.eta_mean, self.eta_std
        self.eta_mean = new_mean
        self.eta_std = max(new_std, 1e-6)
        # print(f"Old eta_mean: {old_eta_mean:.6f}, New eta_mean: {self.eta_mean:.6f}, "
            #   f"Eta_sample: {eta_sample:.6f}, Price: {price}, Demand: {demand}, "
            #   f"Likelihood STD: {likelihood_std:.6f}")


    def thompson_sampling(self):
        sampled_eta = np.random.normal(self.eta_mean, self.eta_std)
        self.sampled_etas.append(sampled_eta)
        return sampled_eta

    def get_demand(self, price, eta):
        return self.a * price ** (-np.abs(eta))

    def calculate_rmse(self):
        estimated_prices = [self.get_price(d, e) for d, e in zip(self.actual_prices, self.sampled_etas)]
        return np.sqrt(mean_squared_error(self.actual_prices, estimated_prices))

    def get_price(self, demand, eta):
        price = (demand / self.a) ** (-1 / np.abs(eta))
        self.price_points.append(price)
        return price

    def calculate_revenue(self, price, demand):
        return price * demand

    def simulate_pricing_with_revenue_tracking(self, data):
        estimated_etas = []
        estimated_demands = []

        for index, row in data.iterrows():
            sampled_eta = self.thompson_sampling()
            demand = self.get_demand(row['price'], sampled_eta)
            estimated_price = self.get_price(demand, sampled_eta)
            revenue = self.calculate_revenue(row['price'], demand)  # Use actual price for revenue calculation
            self.cumulative_revenue += revenue
            self.revenue_history.append(self.cumulative_revenue)
            
            self.actual_prices.append(row['price'])
            self.eta_points.append(sampled_eta)
            
            # Update the posterior with the actual demand observed
            self.update_posterior(row['price'], estimated_price, demand)
            
            # Append the estimated values to the lists
            estimated_etas.append(sampled_eta)
            estimated_demands.append(demand)
            
            if index % 100 == 0 and index > 0:
                print(f'Index: {index}, RMSE: {self.calculate_rmse()}, Cumulative Revenue: {self.cumulative_revenue}, Eta: {self.eta_mean:.4f}')
        
        # Add the estimated values as new columns to the original dataframe
        data['estimated_eta'] = estimated_etas
        data['estimated_demand'] = estimated_demands
        
        return data  # Return the updated DataFrame
    

    def plot_cumulative_revenue_over_time(self):
        plt.figure(figsize=(10, 6))
        plt.plot(self.revenue_history, label='Cumulative Revenue over time')
        plt.xlabel('Time')
        plt.ylabel('Cumulative Revenue')
        plt.legend()
        plt.grid(True)
        plt.show()
        
    def plot_demand_curve(self, index):
        if index < 100 or index >= len(self.sampled_etas):
            print("Index out of range for plotting.")
            return
        plt.figure(figsize=(10, 6))
        plt.scatter(self.price_points[:index], [self.get_demand(p, e) for p, e in zip(self.price_points[:index], self.eta_points[:index])], color='gray', s=10)
        price_range = np.linspace(0.5 * min(self.price_points), 1.5 * max(self.price_points), 100)
        demand_curve = self.get_demand(price_range, self.sampled_etas[index-1])
        plt.plot(price_range, demand_curve, label=f'η: {self.sampled_etas[index-1]:.4f}')
        plt.xlabel('Price')
        plt.ylabel('Demand')

        #  fixed x and y
        plt.xlim([0, max(self.price_points)])
        plt.ylim([0, max([self.get_demand(p, self.eta_mean) for p in price_range])])
        plt.legend()
        plt.grid(True)
        plt.show()


In [ ]:
nan_product_id = '8cf7e821-f0d3-49c6-8eba-e679c0ebcf6a'

# Drop rows where product_id is '8cf7e821-f0d3-49c6-8eba-e679c0ebcf6a'
cleaned_cab_rides_df = cab_rides_df[cab_rides_df['product_id'] != nan_product_id]

# Confirm that rows with the specified product_id have been removed
remaining_missing_values = cleaned_cab_rides_df.isnull().sum()
remaining_missing_values[remaining_missing_values > 0]


Series([], dtype: int64)

In [ ]:
# Function to apply DynamicPricing to a single group
def apply_dynamic_pricing(group, initial_eta_mean, initial_eta_std, a_value):
    # Initialize the DynamicPricing class with given parameters
    dp = DynamicPricing(initial_eta_mean, initial_eta_std, a_value)
    
    # Apply the simulate_pricing_with_revenue_tracking method
    # Assuming that 'price' column exists in the group DataFrame
    group = dp.simulate_pricing_with_revenue_tracking(group)
    
    return group

# Placeholder values for initial parameters (these could be customized)
initial_eta_mean = 0.5  # Example mean value for eta
initial_eta_std = 0.1   # Example standard deviation for eta
a_value = 13            # Example 'a' value for the demand function, this can be adjusted

# Reapply the DynamicPricing to the cleaned data
# Group the cleaned data by 'product_id' and 'source'
grouped_cleaned = cleaned_cab_rides_df.groupby(['product_id', 'source'])

# Apply the DynamicPricing to each group in the cleaned dataset
# Note: This operation could take a long time depending on the size of the dataset
result_dfs_cleaned = [apply_dynamic_pricing(group, initial_eta_mean, initial_eta_std, a_value) for _, group in grouped_cleaned]

# Concatenate the resulting dataframes
result_df_cleaned = pd.concat(result_dfs_cleaned, ignore_index=True)

# Check the first few rows of the result
result_df_cleaned.head()



Index: 263900, RMSE: 14.683778797563466, Cumulative Revenue: 79143.54707505715, Eta: 0.5029
Index: 278300, RMSE: 14.650511182178148, Cumulative Revenue: 84346.53602208069, Eta: 0.5029
Index: 302200, RMSE: 14.624534007349391, Cumulative Revenue: 91192.37513695887, Eta: 0.5029
Index: 311500, RMSE: 14.656654058784392, Cumulative Revenue: 93706.38892180307, Eta: 0.5029
Index: 319400, RMSE: 14.67265043680298, Cumulative Revenue: 96169.97944402839, Eta: 0.5029
Index: 327600, RMSE: 14.680817486122304, Cumulative Revenue: 99057.30271351742, Eta: 0.5029
Index: 420500, RMSE: 14.616317692771766, Cumulative Revenue: 127961.83898748951, Eta: 0.5029
Index: 431000, RMSE: 14.620153234594879, Cumulative Revenue: 131396.0259487698, Eta: 0.5029
Index: 436800, RMSE: 14.61255184658861, Cumulative Revenue: 132958.97199747458, Eta: 0.5029
Index: 444700, RMSE: 14.605851098634988, Cumulative Revenue: 134961.442409801, Eta: 0.5029
Index: 457100, RMSE: 14.595736465615337, Cumulative Revenue: 137874.29254204105, 

,distance,cab_type,time_stamp,destination,source,price,surge_multiplier,id,product_id,name,estimated_eta,estimated_demand
0,2.32,Uber,1545072610662,North End,Back Bay,9.5,1.0,88c309eb-4f98-4410-bf60-2d2f1113abe2,55c66225-fbe7-4fd5-9072-eab1ece5e23e,UberX,0.406508,5.205843
1,1.40,Uber,1545098110455,Boston University,Back Bay,7.5,1.0,0b50bd4f-188d-430a-b934-cfad0bc27199,55c66225-fbe7-4fd5-9072-eab1ece5e23e,UberX,0.506889,4.681497
2,3.45,Uber,1543618688417,South Station,Back Bay,10.5,1.0,3c9ac81a-90ee-4841-a0b6-f35cef745853,55c66225-fbe7-4fd5-9072-eab1ece5e23e,UberX,0.383925,5.270888
3,1.08,Uber,1543622889918,Northeastern University,Back Bay,12.5,1.0,5a61910c-638e-471a-8c5f-a7b377207c6d,55c66225-fbe7-4fd5-9072-eab1ece5e23e,UberX,0.399211,4.742908
4,1.34,Uber,1543308501921,Fenway,Back Bay,7.5,1.0,4066a6b5-53bd-4e79-a8e0-703140f0d74e,55c66225-fbe7-4fd5-9072-eab1ece5e23e,UberX,0.427370,5.495007


In [ ]:
result_df_cleaned.head()

,distance,cab_type,time_stamp,destination,source,price,surge_multiplier,id,product_id,name,estimated_eta,estimated_demand
0,2.32,Uber,1545072610662,North End,Back Bay,9.5,1.0,88c309eb-4f98-4410-bf60-2d2f1113abe2,55c66225-fbe7-4fd5-9072-eab1ece5e23e,UberX,0.406508,5.205843
1,1.40,Uber,1545098110455,Boston University,Back Bay,7.5,1.0,0b50bd4f-188d-430a-b934-cfad0bc27199,55c66225-fbe7-4fd5-9072-eab1ece5e23e,UberX,0.506889,4.681497
2,3.45,Uber,1543618688417,South Station,Back Bay,10.5,1.0,3c9ac81a-90ee-4841-a0b6-f35cef745853,55c66225-fbe7-4fd5-9072-eab1ece5e23e,UberX,0.383925,5.270888
3,1.08,Uber,1543622889918,Northeastern University,Back Bay,12.5,1.0,5a61910c-638e-471a-8c5f-a7b377207c6d,55c66225-fbe7-4fd5-9072-eab1ece5e23e,UberX,0.399211,4.742908
4,1.34,Uber,1543308501921,Fenway,Back Bay,7.5,1.0,4066a6b5-53bd-4e79-a8e0-703140f0d74e,55c66225-fbe7-4fd5-9072-eab1ece5e23e,UberX,0.427370,5.495007


In [ ]:
result_df_cleaned.to_csv('cab_rides_with_eta_and_demand.csv')

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=31c7db23-e6da-4465-a558-ea9a7638bbc9' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>